# Team Members:
## Abhishek Aditya BS - PES1UG19CS019
## Vishal R - PES1UG19CS571

# Towards Deeper Graph Neural Networks
*Liu et al. KDD2020*

Graph neural networks have shown significant success in the field of graph representation learning. Graph convolutions perform neighborhood aggregation and represent one of the most important graph operations. Nevertheless, one layer of these neighborhood aggregation methods only consider immediate neighbors, and the performance decreases when going deeper to enable larger receptive fields. Several recent studies attribute this performance deterioration to the over-smoothing issue, which states that repeated propagation makes node representations of different classes indistinguishable. The authors studied this observation systematically and developed new insights towards deeper graph neural networks. First, they proposed a systematical analysis on this issue and argue that the key factor compromising the performance significantly is the entanglement of representation transformation and propagation in current graph convolution operations. After decoupling these two operations, deeper graph neural networks can be used to learn graph node representations from larger receptive fields. They further provide a theoretical analysis of the above observation when building very deep models, which can serve as a rigorous and gentle description of the over-smoothing issue. Based on their theoretical and empirical analysis, they propose Deep Adaptive Graph Neural Network (DAGNN) to adaptively incorporate information from large receptive fields. A set of experiments on citation, co-authorship, and co-purchase datasets have confirmed our analysis and insights and demonstrated the superiority of our proposed methods.

# Import all the required libraries

In [ ]:
!pip install dgl==0.6.1
!pip install torch==1.9.1
!pip install dgl-cu100

import random
import argparse
import numpy as np
from tqdm import trange

import torch
from torch import nn
from torch.nn import functional as F
from torch.nn import Parameter

import dgl.function as fn
from dgl.data import CoraGraphDataset, CiteseerGraphDataset, PubmedGraphDataset

     |████████████████████████████████| 4.4 MB 11.4 MB/s 
     |████████████████████████████████| 831.4 MB 6.6 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.9.1 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.9.1 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.9.1 which is incompatible.
     |████████████████████████████████| 24.6 MB 1.5 MB/s 


DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [ ]:
class DAGNNConv(nn.Module):
    def __init__(self,
                 in_dim,
                 k):
        super(DAGNNConv, self).__init__()

        self.s = Parameter(torch.FloatTensor(in_dim, 1))
        self.k = k

        self.reset_parameters()

    def reset_parameters(self):
        gain = nn.init.calculate_gain('sigmoid')
        nn.init.xavier_uniform_(self.s, gain=gain)

    def forward(self, graph, feats):

        with graph.local_scope():
            results = [feats]

            degs = graph.in_degrees().float()
            norm = torch.pow(degs, -0.5)
            norm = norm.to(feats.device).unsqueeze(1)

            for _ in range(self.k):
                feats = feats * norm
                graph.ndata['h'] = feats
                graph.update_all(fn.copy_u('h', 'm'),
                                 fn.sum('m', 'h'))
                feats = graph.ndata['h']
                feats = feats * norm
                results.append(feats)

            H = torch.stack(results, dim=1)
            S = F.sigmoid(torch.matmul(H, self.s))
            S = S.permute(0, 2, 1)
            H = torch.matmul(S, H).squeeze()

            return H

In [ ]:
class MLPLayer(nn.Module):
    def __init__(self,
                 in_dim,
                 out_dim,
                 bias=True,
                 activation=None,
                 dropout=0):
        super(MLPLayer, self).__init__()

        self.linear = nn.Linear(in_dim, out_dim, bias=bias)
        self.activation = activation
        self.dropout = nn.Dropout(dropout)
        self.reset_parameters()

    def reset_parameters(self):
        gain = 1.
        if self.activation is F.relu:
            gain = nn.init.calculate_gain('relu')
        nn.init.xavier_uniform_(self.linear.weight, gain=gain)
        if self.linear.bias is not None:
            nn.init.zeros_(self.linear.bias)

    def forward(self, feats):

        feats = self.dropout(feats)
        feats = self.linear(feats)
        if self.activation:
            feats = self.activation(feats)

        return feats

In [ ]:
class DAGNN(nn.Module):
    def __init__(self,
                 k,
                 in_dim,
                 hid_dim,
                 out_dim,
                 bias=True,
                 activation=F.relu,
                 dropout=0, ):
        super(DAGNN, self).__init__()
        self.mlp = nn.ModuleList()
        self.mlp.append(MLPLayer(in_dim=in_dim, out_dim=hid_dim, bias=bias,
                                 activation=activation, dropout=dropout))
        self.mlp.append(MLPLayer(in_dim=hid_dim, out_dim=out_dim, bias=bias,
                                 activation=None, dropout=dropout))
        self.dagnn = DAGNNConv(in_dim=out_dim, k=k)

    def forward(self, graph, feats):
        for layer in self.mlp:
            feats = layer(feats)
        feats = self.dagnn(graph, feats)
        return feats

In [ ]:
def evaluate(model, graph, feats, labels, idxs):
    model.eval()
    with torch.no_grad():
        logits = model(graph, feats)
        results = ()
        for idx in idxs:
            loss = F.cross_entropy(logits[idx], labels[idx])
            acc = torch.sum(logits[idx].argmax(dim=1) == labels[idx]).item() / len(idx)
            results += (loss, acc)
    return results


def generate_random_seeds(seed, nums):
    random.seed(seed)
    return [random.randint(1, 999999999) for _ in range(nums)]


def set_random_state(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

In [ ]:
args={"dataset":'Cora', "gpu":0, "runs":100, "epochs":1500, "early-stopping":100, "lr":0.01, "lamb":0.005, "k":12, "hid-dim":64, "dropout":0.8}

In [ ]:
def main(args):
    # Step 1: Prepare graph data and retrieve train/validation/test index ============================= #
    # Load from DGL dataset
    if args["dataset"] == 'Cora':
        dataset = CoraGraphDataset()
    elif args["dataset"] == 'Citeseer':
        dataset = CiteseerGraphDataset()
    elif args["dataset"] == 'Pubmed':
        dataset = PubmedGraphDataset()
    else:
        raise ValueError('Dataset {} is invalid.'.format(args["dataset"]))

    graph = dataset[0]
    graph = graph.add_self_loop()

    # check cuda
    if args["gpu"] >= 0 and torch.cuda.is_available():
        device = 'cuda:{}'.format(args["gpu"])
    else:
        device = 'cpu'

    # retrieve the number of classes
    n_classes = dataset.num_classes

    # retrieve labels of ground truth
    labels = graph.ndata.pop('label').to(device).long()

    # Extract node features
    feats = graph.ndata.pop('feat').to(device)
    n_features = feats.shape[-1]

    # retrieve masks for train/validation/test
    train_mask = graph.ndata.pop('train_mask')
    val_mask = graph.ndata.pop('val_mask')
    test_mask = graph.ndata.pop('test_mask')

    train_idx = torch.nonzero(train_mask, as_tuple=False).squeeze().to(device)
    val_idx = torch.nonzero(val_mask, as_tuple=False).squeeze().to(device)
    test_idx = torch.nonzero(test_mask, as_tuple=False).squeeze().to(device)

    graph = graph.to(device)

    # Step 2: Create model =================================================================== #
    model = DAGNN(k=args["k"],
                  in_dim=n_features,
                  hid_dim=args["hid-dim"],
                  out_dim=n_classes,
                  dropout=args["dropout"])
    model = model.to(device)

    # Step 3: Create training components ===================================================== #
    loss_fn = F.cross_entropy
    opt = torch.optim.Adam(model.parameters(), lr=args["lr"], weight_decay=args["lamb"])

    # Step 4: training epochs =============================================================== #
    loss = float('inf')
    best_acc = 0
    no_improvement = 0
    epochs = trange(args["epochs"], desc='Accuracy & Loss')

    for _ in epochs:
        model.train()

        logits = model(graph, feats)

        # compute loss
        train_loss = loss_fn(logits[train_idx], labels[train_idx])

        # backward
        opt.zero_grad()
        train_loss.backward()
        opt.step()

        train_loss, train_acc, valid_loss, valid_acc, test_loss, test_acc = evaluate(model, graph, feats, labels,
                                                                                     (train_idx, val_idx, test_idx))

        # Print out performance
        epochs.set_description('Train Acc {:.4f} | Train Loss {:.4f} | Val Acc {:.4f} | Val loss {:.4f}'.format(
            train_acc, train_loss.item(), valid_acc, valid_loss.item()))

        if valid_loss > loss:
            no_improvement += 1
            if no_improvement == args["early-stopping"]:
                print('Early stop.')
                break
        else:
            no_improvement = 0
            loss = valid_loss
            best_acc = test_acc

    print("Test Acc {:.4f}".format(best_acc))
    return best_acc


if __name__ == "__main__":

    acc_lists = []
    random_seeds = generate_random_seeds(seed=1222, nums=args["runs"])

    for run in range(args["runs"]):
        set_random_state(random_seeds[run])
        acc_lists.append(main(args))

    acc_lists = np.array(acc_lists)

    mean = np.around(np.mean(acc_lists, axis=0), decimals=4)
    std = np.around(np.std(acc_lists, axis=0), decimals=4)

    print('Total acc: ', acc_lists)
    print('mean', mean)
    print('std', std)

Loading from cache failed, re-processing.
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


Accuracy & Loss:   0%|          | 0/1500 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
Train Acc 0.9857 | Train Loss 0.1707 | Val Acc 0.7960 | Val loss 0.6875:  29%|██▉       | 437/1500 [00:11<00:27, 38.98it/s]


Early stop.
Test Acc 0.8480
Loading from cache failed, re-processing.
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


Accuracy & Loss:   0%|          | 0/1500 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
Train Acc 0.9857 | Train Loss 0.1743 | Val Acc 0.7940 | Val loss 0.6620:  16%|█▋        | 244/1500 [00:05<00:28, 43.87it/s]


Early stop.
Test Acc 0.8390
Loading from cache failed, re-processing.
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


...

Early stop.
Test Acc 0.8400
Loading from cache failed, re-processing.
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


Accuracy & Loss:   0%|          | 0/1500 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
Train Acc 0.9714 | Train Loss 0.1910 | Val Acc 0.8020 | Val loss 0.6706:  17%|█▋        | 251/1500 [00:06<00:30, 41.62it/s]


Early stop.
Test Acc 0.8500
Loading from cache failed, re-processing.
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


Accuracy & Loss:   0%|          | 0/1500 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
Train Acc 1.0000 | Train Loss 0.1714 | Val Acc 0.8020 | Val loss 0.6613:  33%|███▎      | 494/1500 [00:11<00:24, 41.37it/s]


Early stop.
Test Acc 0.8480
Total acc:  [0.848 0.839 0.844 0.843 0.833 0.84  0.83  0.844 0.835 0.846 0.852 0.846
 0.841 0.844 0.847 0.846 0.845 0.841 0.845 0.847 0.84  0.843 0.839 0.837
 0.84  0.842 0.844 0.844 0.848 0.844 0.841 0.844 0.847 0.844 0.843 0.849
 0.846 0.837 0.837 0.84  0.837 0.848 0.838 0.84  0.854 0.846 0.849 0.848
 0.847 0.844 0.842 0.857 0.846 0.846 0.844 0.854 0.846 0.842 0.846 0.852
 0.848 0.845 0.843 0.84  0.847 0.84  0.846 0.85  0.844 0.839 0.847 0.847
 0.849 0.849 0.84  0.843 0.855 0.844 0.84  0.839 0.844 0.85  0.835 0.839
 0.851 0.852 0.842 0.856 0.835 0.842 0.841 0.836 0.839 0.839 0.846 0.847
 0.84  0.84  0.85  0.848]
mean 0.844
std 0.0051
